In [58]:
import json, sqlite3, csv


In [74]:

json_file = "/Users/akshayshewani/Documents/AiSPRY_ELT/data/customers.json"
csv_file = "/Users/akshayshewani/Documents/AiSPRY_ELT/data/orders_12_20.csv"

sqlite_keywords = set([
        "ABORT", "ACTION", "ADD", "AFTER", "ALL", "ALTER", "ANALYZE", "AND", "AS", "ASC", "ATTACH", "AUTOINCREMENT",
        "BEFORE", "BEGIN", "BETWEEN", "BY", "CASCADE", "CASE", "CAST", "CHECK", "COLLATE", "COLUMN", "COMMIT",
        "CONFLICT", "CONSTRAINT", "CREATE", "CROSS", "CURRENT_DATE", "CURRENT_TIME", "CURRENT_TIMESTAMP", "DATABASE",
        "DEFAULT", "DEFERRABLE", "DEFERRED", "DELETE", "DESC", "DETACH", "DISTINCT", "DROP", "EACH", "ELSE", "END",
        "ESCAPE", "EXCEPT", "EXCLUSIVE", "EXISTS", "EXPLAIN", "FAIL", "FOR", "FOREIGN", "FROM", "FULL", "GLOB", 
        "GROUP", "HAVING", "IF", "IGNORE", "IMMEDIATE", "IN", "INDEX", "INDEXED", "INITIALLY", "INNER", "INSERT", 
        "INSTEAD", "INTERSECT", "INTO", "IS", "ISNULL", "JOIN", "KEY", "LEFT", "LIKE", "LIMIT", "MATCH", "NATURAL", 
        "NO", "NOT", "NOTNULL", "NULL", "OF", "OFFSET", "ON", "OR", "ORDER", "OUTER", "PLAN", "PRAGMA", "PRIMARY", 
        "QUERY", "RAISE", "RECURSIVE", "REFERENCES", "REGEXP", "REINDEX", "RELEASE", "RENAME", "REPLACE", "RESTRICT", 
        "RIGHT", "ROLLBACK", "ROW", "SAVEPOINT", "SELECT", "SET", "TABLE", "TEMP", "TEMPORARY", "THEN", "TO", 
        "TRANSACTION", "TRIGGER", "UNION", "UNIQUE", "UPDATE", "USING", "VACUUM", "VALUES", "VIEW", "VIRTUAL", 
        "WHEN", "WHERE", "WITH", "WITHOUT"
    ])
    

In [78]:


def clean_columns(columns, sqlite_keywords):

    # Process columns: replace spaces with underscores and avoid SQLite keywords
    renamed_columns = []
    for col in columns:
        # Replace spaces with underscores
        new_col = col.replace(' ', '_')
        # If the column is a reserved keyword, append '_col' to the name
        if new_col.upper() in sqlite_keywords:
            new_col = new_col + '_col'
        renamed_columns.append(new_col)
    
    return renamed_columns


['E_Mail', 'Select_col', 'Customer_Since', 'Place_Name', 'Where_col']


In [60]:
def clean_json(json_file):
    
    json_objects = []
    
    with open(json_file, 'r') as file:
        for line in file:
            line = line.strip()  # Remove leading/trailing whitespaces
            if line:  # Ensure the line isn't empty
                try:
                    json_objects.append(json.loads(line))
                except json.JSONDecodeError as e:
                    print(f"Error decoding JSON: {e}")
    
    return json_objects

In [84]:
def clean_csv(csv_file):

    csv_data = []

    with open(csv_file, 'r') as file:
        reader = csv.DictReader(file)  # Reads each row as a dictionary

        for row in reader:
            csv_data.append(row)  # Append each row (as a dict) to the list
    
    return csv_data

In [88]:
def load_data_to_sqlite(data, table_name, db_file):
  
    # Connect to the database
    conn = sqlite3.connect(db_file)
    c = conn.cursor()

    # Get the column names from the first dictionary
    columns = list(data[0].keys())
    
    columns = clean_columns(columns, sqlite_keywords)

    drop_table_query = f"DROP TABLE IF EXISTS {table_name}"
    c.execute(drop_table_query)

    # Create the table with new column names
    create_table_query = f"CREATE TABLE {table_name} ({', '.join([f'{col} TEXT' for col in columns])})"
    c.execute(create_table_query)

    # Insert data into the table
    insert_query = f"INSERT INTO {table_name} ({', '.join(columns)}) VALUES ({', '.join(['?' for _ in columns])})"
    for row in data:
        # print(insert_query)
        # print(row.values())
        
        c.execute(insert_query, tuple(row.values()))
        

    # Commit changes and close the connection
    conn.commit()
    conn.close()

    

In [80]:
csv_object = clean_csv(csv_file)

In [81]:
csv_object_test = csv_object[:10]

In [55]:
json_object = clean_json(json_file)
db_file = "./test.db"

In [56]:
json_object_test = json_object[:10]

In [83]:
#csv_object_test

In [89]:
load_data_to_sqlite(csv_object_test, 'orders', db_file)

In [90]:
load_data_to_sqlite(json_object_test, 'customers', db_file)

In [13]:
import json

def load_malformed_json(json_file):
    """
    Reads a JSON file line by line and parses each line as a separate JSON object.
    Useful for fixing JSONDecodeError due to improperly formatted JSON.
    
    Parameters:
    json_file (str): Path to the malformed JSON file.
    
    Returns:
    List[dict]: A list of parsed JSON objects.
    """
    json_objects = []
    
    with open(json_file, 'r') as file:
        for line in file:
            line = line.strip()  # Remove leading/trailing whitespaces
            if line:  # Ensure the line isn't empty
                try:
                    json_objects.append(json.loads(line))
                except json.JSONDecodeError as e:
                    print(f"Error decoding JSON: {e}")
    
    return json_objects

# Example usage
json_file = 'malformed_data.json'
data = load_malformed_json(json_file)

print(data)  # List of parsed JSON objects


In [87]:
json_object

[{'cust_id': 60124.0,
  'Gender': 'F',
  'age': 43.0,
  'full_name': 'Titus, Jani',
  'E Mail': 'jani.titus@gmail.com',
  'Customer Since': '8/22/2006',
  'Place Name': 'Vinson',
  'County': 'Harmon',
  'City': 'Vinson',
  'State': 'OK',
  'Zip': 73571,
  'Region': 'South'},
 {'cust_id': 42485.0,
  'Gender': 'M',
  'age': 28.0,
  'full_name': 'Eaker, Lee',
  'E Mail': 'lee.eaker@gmail.com',
  'Customer Since': '2/4/1981',
  'Place Name': 'Graham',
  'County': 'Bradford',
  'City': 'Graham',
  'State': 'FL',
  'Zip': 32042,
  'Region': 'South'},
 {'cust_id': 53620.0,
  'Gender': 'M',
  'age': 65.0,
  'full_name': 'Simoneau, Jason',
  'E Mail': 'jason.simoneau@gmail.com',
  'Customer Since': '6/27/2010',
  'Place Name': 'Grand Forks',
  'County': 'Grand Forks',
  'City': 'Grand Forks',
  'State': 'ND',
  'Zip': 58201,
  'Region': 'Midwest'},
 {'cust_id': 56836.0,
  'Gender': 'M',
  'age': 33.0,
  'full_name': 'Bayless, Grover',
  'E Mail': 'grover.bayless@yahoo.com',
  'Customer Since': 